In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as f
from torch.utils.data import DataLoader
import math


In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = set(stopwords.words('english'))

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import classification_report, confusion_matrix

import os
from tqdm import tqdm
tqdm.pandas()
from collections import Counter
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
data = pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
def transform_label(label):
    return 1 if label == 'positive' else 0

data['label'] = data['sentiment'].progress_apply(transform_label)
data.head()

100%|██████████| 50000/50000 [00:00<00:00, 768015.94it/s]


,review,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [22]:
data.sentiment.value_counts()


positive    25000
negative    25000
Name: sentiment, dtype: int64

In [23]:
data['token_length'] = data.review.progress_apply(lambda x: len(x.split()))

100%|██████████| 50000/50000 [00:01<00:00, 32999.13it/s]


In [ ]:
data_pos = data[data['label'] == 1]
data_pos['token_length'].describe()

------------------------------------

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
torch.manual_seed(125)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(125)

In [ ]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (1.0,))])

from torchvision.datasets import MNIST

download_root = '../Mnist_dataset'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 82223936.49it/s]


Extracting ../Mnist_dataset/MNIST/raw/train-images-idx3-ubyte.gz to ../Mnist_dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 49362548.42it/s]


Extracting ../Mnist_dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ../Mnist_dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26560150.69it/s]


Extracting ../Mnist_dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ../Mnist_dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3279485.07it/s]


Extracting ../Mnist_dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../Mnist_dataset/MNIST/raw



In [1]:
!pip install 'portalocker>=2.0.0'
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')

def tokenize(label, line):
    return line.split()

tokens = []
for label, line in train_iter:
    tokens += tokenize(label, line)

ModuleNotFoundError: ignored

In [ ]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters/(len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

In [ ]:
class LSTMCell(nn.Module):
  def __init__(self, input_size, hidden_size, bias=True):
    super(LSTMCell, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias
    self.x2h = nn.Linear(input_size, 4*hidden_size, bias=bias)
    self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias=bias)
    self.reset_parameters()

  def reset_parameters(self):
    std = 1.0 / math.sqrt(self.hidden_size)
    for w in self.parameters():
      w.data.uniform_(-std, std)
  
  def forward(self, x, hidden):
    hx, cx = hidden
    x = x.view(-1, x.size(1))

    gates = self.x2h(x) + self.h2h(hx)
    gats = gates.squeeze()
    ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

    ingate = f.sigmoid(ingate)
    forgetgate = f.sigmoid(forgetgate)
    cellgate = f.tanh(cellgate)
    outgate = f.sigmoid(outgate)

    cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
    hy = torch.mul(outgate, f.tanh(cy))

    return (hy, cy)


In [ ]:
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
    super(LSTMModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim
    self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    if torch.cuda.is_available():
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)).cuda()
    else:
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

    if torch.cuda.is_available():
      c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)).cuda()
    else:
      c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

    outs = []
    cn = c0[0,:,:]
    hn = h0[0,:,:]

    for seq in range(x.size(1)):
      hn, cn = self.lstm(x[:, seq, :], (hn, cn))
      outs.append(hn)

    out = outs[-1].squeeze()
    out = self.fc(out)
    return out


In [ ]:
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 18

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available():
  model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
seq_dim=28
loss_list = []
iter = 0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    if torch.cuda.is_available():
      images = Variable(images.view(-1, seq_dim, input_dim).cuda())
      labels = Variable(labels.cuda())

    else :
      images = Variable(images.view(-1, seq_dim, input_dim))
      labels = Variable(labels)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    if torch.cuda.is_available():
      loss.cuda()

    loss.backward()
    optimizer.step()
    loss_list.append(loss.item())
    iter += 1

    if iter % 500 == 0:
      correct = 0
      total = 0
      
      for images, labels in valid_loader:
        images = Variable(images.view(-1, seq_dim, input_dim))

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum()

      accuracy = 100 * correct / total
      print(f'Iter : {iter:4d} Loss : {loss.item():.9f} Accuracy : {accuracy}')
  

Iter :  500 Loss : 0.003988761 Accuracy : 98.69000244140625
Iter : 1000 Loss : 0.003604538 Accuracy : 98.73999786376953
Iter : 1500 Loss : 0.006367204 Accuracy : 98.62999725341797
Iter : 2000 Loss : 0.011742361 Accuracy : 98.7300033569336
Iter : 2500 Loss : 0.020207359 Accuracy : 98.4800033569336
Iter : 3000 Loss : 0.005296821 Accuracy : 98.62999725341797
Iter : 3500 Loss : 0.000428752 Accuracy : 98.63999938964844
Iter : 4000 Loss : 0.004063338 Accuracy : 98.66999816894531
Iter : 4500 Loss : 0.001952233 Accuracy : 98.70999908447266
Iter : 5000 Loss : 0.007456235 Accuracy : 98.66999816894531
Iter : 5500 Loss : 0.003253677 Accuracy : 98.66999816894531
Iter : 6000 Loss : 0.001593741 Accuracy : 98.69999694824219
Iter : 6500 Loss : 0.070519656 Accuracy : 98.54000091552734
Iter : 7000 Loss : 0.008630849 Accuracy : 98.54000091552734
Iter : 7500 Loss : 0.023352435 Accuracy : 98.62999725341797
Iter : 8000 Loss : 0.002629545 Accuracy : 98.43000030517578
Iter : 8500 Loss : 0.009031124 Accuracy : 

In [ ]:
def evaluate(model, val_iter):
  corrects, total, total_loss = 0, 0, 0
  model.eval()

  for images, labels in val_iter:
    images = Variable(images.view(-1, seq_dim, input_dim).to(device))
    logit = model(images).to(device)
    loss = f.cross_entropy(logit, labels, reduction='sum')
    _, predicted = torch.max(logit.data, 1)
    total += labels.size(0)
    total_loss += loss.item()
    corrects += (predicted == labels).sum()

  avg_loss = total_loss / len(val_iter.dataset)
  avg_accuracy = corrects / total
  return avg_loss, avg_accuracy

In [ ]:
test_loss, test_acc = evaluate(model, test_loader)
print(f"Test Loss : {test_loss:5.2f} | Test Accuracy : {test_acc:5.2f}")

Test Loss :  0.06 | Test Accuracy :  0.98
